
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-repo/your-notebook.ipynb)


In [ ]:
# Install necessary libraries
!pip install networkx nx_arangodb cugraph-cu12 cudf-cu12 openai langchain_community python-telegram-bot matplotlib

In [ ]:
import io
import os
import networkx as nx
import nx_arangodb as nxadb
import torch
import cudf
import cugraph
from networkx.algorithms import community

from arango import ArangoClient
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import re

from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_community.graphs import ArangoGraph
from langchain_community.chains.graph_qa.arangodb import ArangoGraphQAChain
from langchain_core.tools import tool

import openai
from openai import AzureOpenAI


In [ ]:
# Input your own key Load database credentials securely 
os.environ["DATABASE_HOST"] = "https://tutorials.arangodb.cloud:8529"
os.environ["DATABASE_USERNAME"] = "TUT8ipacx2t02ebj95v26ioo"
os.environ["DATABASE_PASSWORD"] = "TUTb69rmmvlqygt7dceaxf8d"
os.environ["DATABASE_NAME"] = "TUTnhcypojdm6nocid8h4x3v"

# OpenAI API details
# uncomment openai and comment azure openai if you are using openai api 
# openai_api_key = "s***************"  # Replace with your OpenAI API key


# Azure OpenAI API details
api_base = "https://thisisoajo.openai.azure.com/"  # Replace with your Azure OpenAI resource URL
model = "gpt-4o"
api_key = "9I4UEJweVUdih04Uv8AXcAxs5H8jSQRfwaugcSQYHcI882wSpFvqJQQJ99BAACL93NaXJ3w3AAABACOGkv4f"
api_version = "2023-06-01-preview"

# Load vulnerability data from CSV
def load_vulnerabilities():
    data = """cve_id,mod_date,pub_date,cvss,cwe_code,cwe_name,summary
    CVE-2019-16548,2019-11-21 15:15:00,2019-11-21 15:15:00,6.8,352,Cross-Site Request Forgery (CSRF),A cross-site request forgery vulnerability in Jenkins Google Compute Engine Plugin 4.1.1 and earlier.
    CVE-2019-16547,2019-11-21 15:15:00,2019-11-21 15:15:00,4.0,732,Incorrect Permission Assignment for Critical Resource,Missing permission checks in various API endpoints.
    CVE-2019-16546,2019-11-21 15:15:00,2019-11-21 15:15:00,4.3,639,Authorization Bypass Through User-Controlled Key,Jenkins Google Compute Engine Plugin 4.1.1 and earlier does not verify SSH host keys.
    CVE-2013-2092,2019-11-20 21:22:00,2019-11-20 21:15:00,4.3,79,Improper Neutralization of Input During Web Page Generation ('Cross-site Scripting'),Cross-site Scripting (XSS) in Dolibarr ERP/CRM 3.3.1.
    """
    # df = pd.read_csv(pd.compat.StringIO(data))
    df = pd.read_csv(io.StringIO(data))
    return df



In [ ]:
# Create a vulnerability graph
def create_vulnerability_graph(df):
    G = nx.Graph()
    for _, row in df.iterrows():
        G.add_node(row['cve_id'], severity=row['cvss'], cwe=row['cwe_name'], summary=row['summary'])
    return G

df_vuln = load_vulnerabilities()
G_nx = create_vulnerability_graph(df_vuln)

# Store the Graph in ArangoDB
G_adb = nxadb.Graph(incoming_graph_data=G_nx, name="VulnerabilityGraphX")

# Analyze graph using PageRank
def analyze_graph(graph):
    if torch.cuda.is_available():
        gdf = cudf.DataFrame(list(graph.edges()), columns=["source", "destination"])
        G_cu = cugraph.Graph()
        G_cu.from_cudf_edgelist(gdf, source="source", destination="destination")
        pagerank_df = cugraph.pagerank(G_cu)
        pagerank_scores = {row["vertex"]: row["pagerank"] for row in pagerank_df.to_pandas().to_dict("records")}
    else:
        pagerank_scores = nx.pagerank(graph)
    return pagerank_scores

pagerank_scores = analyze_graph(G_nx)
for node, score in pagerank_scores.items():
    if node in G_adb.nodes:
        G_adb.nodes[node]["pagerank"] = score

# Define LLM for queries
llm = ChatOpenAI(temperature=0, model_name="gpt-4o", openai_api_key=api_key)

def chat_with_gpt(prompt):
    """Interact with GPT-4o via Azure OpenAI API."""
    client = AzureOpenAI(
        api_key=api_key,
        api_version=api_version,
        base_url=f"{api_base}/openai/deployments/{model}"
    )

    client = AzureOpenAI(
        api_key=api_key,
        api_version=api_version,
        base_url=f"{api_base}/openai/deployments/{model}"
    )


    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": (
                    "Routes List:\n"
                    "Your roles is "
                )
            },
            {"role": "user", "content": prompt}
        ],
        max_tokens=100,
        temperature=0.7,
    )
    return response.choices[0].message.content.strip()



In [ ]:
# Define a function to query the graph
def query_graph(query):
    """Query the graph using the ArangoGraphQAChain."""
    qa_chain = ArangoGraphQAChain(graph=G_adb, llm=llm)
    response = qa_chain.query(query)
    return response

#   call the function
query = "What are the vulnerabilities in the graph?"

if __name__ == "__main__":
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Goodbye!")
            break
        response = chat_with_gpt(user_input)
        print("GPT-4o:", response)




In [ ]:
# use this below if you are using openAI api directly
# @tool
# def text_to_aql_to_text(query: str):
#     llm_chain = ChatOpenAI(temperature=0, model_name="gpt-4o", openai_api_key=api_key)
#     chain = ArangoGraphQAChain.from_llm(llm=llm_chain, graph=G_adb, verbose=True, allow_dangerous_requests=True)
#     result = chain.invoke(query)
#     return str(result["result"])

# G_adb.chat("Which vulnerability has the highest PageRank value?", allow_dangerous_requests=True)
